In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
import pandas as pd

In [17]:
df1 = pd.read_csv('/content/drive/MyDrive/Task2/all-cancelled-flights.csv')
df2 = pd.read_csv('/content/drive/MyDrive/Task2/cancellation-codes.csv')
print(df1.head())


  FLIGHT_DATE       DAY OP_CARRIER               CARRIER_NAME  \
0  2020-10-01  Thursday         BA   AnyCompany Blue Airlines   
1  2020-10-01  Thursday         BA   AnyCompany Blue Airlines   
2  2020-10-01  Thursday         GA  AnyCompany Green Airlines   
3  2020-10-01  Thursday         GA  AnyCompany Green Airlines   
4  2020-10-01  Thursday         GA  AnyCompany Green Airlines   

   OP_CARRIER_FL_NUM ORIGIN DISPLAY_AIRPORT_NAME_ORIGIN DEST  \
0               5029    ATL           Atlanta Municipal  MLB   
1               5069    MLB          Melbourne Regional  ATL   
2                134    DFW  Dallas Fort Worth Regional  OGG   
3               1960    OGG             Kahului Airport  PHX   
4               2242    LAX   Los Angeles International  KOA   

          DISPLAY_AIRPORT_NAME_DEST CANCELLATION_CODE  NUMBER_OF_FLIGHTS  \
0                Melbourne Regional                 A                  1   
1                 Atlanta Municipal                 A                  1

In [18]:
# 計算CANCELL欄位中各類別的數量
cancel_counts = df1['CANCELLATION_CODE'].value_counts()

# 計算各類別的百分比
cancel_percentage = cancel_counts / cancel_counts.sum() * 100

# 顯示結果
print(cancel_percentage)

CANCELLATION_CODE
B    52.713887
A    37.304273
C     8.708309
D     1.273532
Name: count, dtype: float64


In [19]:
# 計算每個航班班機號的出現次數
flight_counts = df1['OP_CARRIER_FL_NUM'].value_counts()

# 找出出現次數最多的航班班機號
most_frequent_flight = flight_counts.idxmax()
most_frequent_count = flight_counts.max()

# 計算該航班號的占比
total_flights = flight_counts.sum()
most_frequent_percentage = (most_frequent_count / total_flights) * 100

# 顯示結果
print(f"取消最多的航班班機號是: {most_frequent_flight}，取消了 {most_frequent_count} 次，占比為 {most_frequent_percentage:.2f}%")

取消最多的航班班機號是: 76，取消了 153 次，占比為 0.06%


In [25]:
# 確保日期格式正確，將FL_DATE轉換為datetime格式
df1['FLIGHT_DATE'] = pd.to_datetime(df1['FLIGHT_DATE'])

# 篩選出出發地為OGG的航班
ogg_flights = df1[df1['ORIGIN'] == 'OGG']

# 篩選2020/10/01-2020/12/31、2021和2022/01/01-2022/10/31的資料
df1_2020 = ogg_flights[(ogg_flights['FLIGHT_DATE'] >= '2020-10-01') & (ogg_flights['FLIGHT_DATE'] <= '2020-12-31')]
df1_2021 = ogg_flights[(ogg_flights['FLIGHT_DATE'] >= '2021-01-01') & (ogg_flights['FLIGHT_DATE'] <= '2021-12-31')]
df1_2022 = ogg_flights[(ogg_flights['FLIGHT_DATE'] >= '2022-01-01') & (ogg_flights['FLIGHT_DATE'] <= '2022-10-31')]

# 定義函數計算每一年取消航班的總占比及取消原因占比
def calculate_cancellation_stats(dataframe, year):
    # 篩選出已取消的航班
    cancelled_flights = dataframe[dataframe['CANCELLATION_CODE'].notna()]

    # 計算取消航班的占比
    cancelled_percentage = (len(cancelled_flights) / len(dataframe)) * 100 if len(dataframe) > 0 else 0

    # 計算取消原因的占比
    cancel_reason_percentage = cancelled_flights['CANCELLATION_CODE'].value_counts(normalize=True) * 100

    # 顯示結果
    print(f"{year} 年的取消原因占比:")
    print(cancel_reason_percentage)

# 分別計算每一年的結果
calculate_cancellation_stats(df1_2020, 2020)
calculate_cancellation_stats(df1_2021, 2021)
calculate_cancellation_stats(df1_2022, 2022)

2020 年的取消原因占比:
CANCELLATION_CODE
D    87.958115
A    10.994764
B     1.047120
Name: proportion, dtype: float64
2021 年的取消原因占比:
CANCELLATION_CODE
A    91.071429
B     6.696429
D     1.785714
C     0.446429
Name: proportion, dtype: float64
2022 年的取消原因占比:
CANCELLATION_CODE
A    97.340426
B     2.127660
C     0.531915
Name: proportion, dtype: float64


In [28]:
# 篩選出ORIGIN為IAD的航班
iad_flights = df1[df1['ORIGIN'] == 'IAD']

# 找出從IAD起飛次數最多的航班名(OP_CARRIER)
most_frequent_carrier = iad_flights['OP_CARRIER'].value_counts().idxmax()
most_frequent_carrier_count = iad_flights['OP_CARRIER'].value_counts().max()

# 篩選出該航班名的所有航班
carrier_flights = iad_flights[iad_flights['OP_CARRIER'] == most_frequent_carrier]

# 找出該航班名中被取消的航班，並計算取消原因的出現次數
cancelled_carrier_flights = carrier_flights[carrier_flights['CANCELLATION_CODE'].notna()]
most_frequent_cancellation = cancelled_carrier_flights['CANCELLATION_CODE'].value_counts().idxmax()
most_frequent_cancellation_count = cancelled_carrier_flights['CANCELLATION_CODE'].value_counts().max()

# 顯示結果
print(f"從IAD起飛次數最多的航班名是: {most_frequent_carrier}，共 {most_frequent_carrier_count} 次")
if not cancelled_carrier_flights.empty:
    print(f"該航班最常遇到的取消原因是: {most_frequent_cancellation}，出現了 {most_frequent_cancellation_count} 次")
else:
    print(f"該航班從未被取消")

從IAD起飛次數最多的航班名是: AA，共 654 次
該航班最常遇到的取消原因是: B，出現了 289 次


In [33]:
import os
merged_data = pd.merge(df1, df2, left_on='CANCELLATION_CODE', right_on='Code', how='left')
print(merged_data.head())
# 指定保存的路徑 (Google Drive 中的 Task2 資料夾)
output_path = '/content/drive/MyDrive/Task2/merged_cancelled_flights.csv'

# 在保存檔案前，先檢查目錄是否存在，如果不存在則建立
os.makedirs(os.path.dirname(output_path), exist_ok=True)

# 保存合併結果到 Task2 資料夾
merged_data.to_csv(output_path, index=False)

print(f"合併後的檔案已成功保存到: {output_path}")

  FLIGHT_DATE       DAY OP_CARRIER               CARRIER_NAME  \
0  2020-10-01  Thursday         BA   AnyCompany Blue Airlines   
1  2020-10-01  Thursday         BA   AnyCompany Blue Airlines   
2  2020-10-01  Thursday         GA  AnyCompany Green Airlines   
3  2020-10-01  Thursday         GA  AnyCompany Green Airlines   
4  2020-10-01  Thursday         GA  AnyCompany Green Airlines   

   OP_CARRIER_FL_NUM ORIGIN DISPLAY_AIRPORT_NAME_ORIGIN DEST  \
0               5029    ATL           Atlanta Municipal  MLB   
1               5069    MLB          Melbourne Regional  ATL   
2                134    DFW  Dallas Fort Worth Regional  OGG   
3               1960    OGG             Kahului Airport  PHX   
4               2242    LAX   Los Angeles International  KOA   

          DISPLAY_AIRPORT_NAME_DEST CANCELLATION_CODE  NUMBER_OF_FLIGHTS  \
0                Melbourne Regional                 A                  1   
1                 Atlanta Municipal                 A                  1

In [34]:
# 篩選出取消原因是 "National Air System" 或 "Security" 的資料
affected_flights = merged_data[merged_data['Description'].isin(['National Air System', 'Security'])]

# 找出受到這兩種原因取消航班的航空公司
affected_carriers = affected_flights['CARRIER_NAME'].unique()

# 找出未受到這兩種原因取消航班的航空公司
all_carriers = merged_data['CARRIER_NAME'].unique()
unaffected_carriers = [carrier for carrier in all_carriers if carrier not in affected_carriers]

# 計算每個航空公司因為 "National Air System" 或 "Security" 被取消的航班次數
carrier_cancellation_counts = affected_flights['CARRIER_NAME'].value_counts()

# 顯示結果
print("未受到 'National Air System' 和 'Security' 取消航班的航空公司:")
print(unaffected_carriers)

print("\n曾經因 'National Air System' 和 'Security' 取消航班的航空公司以及取消次數:")
print(carrier_cancellation_counts)

未受到 'National Air System' 和 'Security' 取消航班的航空公司:
['AnyCompany Red Airlines']

曾經因 'National Air System' 和 'Security' 取消航班的航空公司以及取消次數:
CARRIER_NAME
AnyCompany Yellow Airlines      3848
AnyCompany Sand Airlines        3617
AnyCompany Blue Airlines        3378
AnyCompany Eclipse Airlines     2747
AnyCompany Green Airlines       2009
AnyCompany Carrot Airlines      1728
AnyCompany Apricot Airlines     1592
AnyCompany Denim Airlines       1520
AnyCompany Pink Airlines        1394
AnyCompany Maroon Airlines       980
AnyCompany Violet Airlines       787
AnyCompany Flint Airlines        708
AnyCompany Harp Airlines         620
AnyCompany Lavender Airlines     198
AnyCompany Indigo Airlines       129
AnyCompany Orange Airlines        85
Name: count, dtype: int64
